In [1]:
"""
Enhanced Railway Infrastructure Mapping and GPS Data Quality Validation
========================================================================

This script implements comprehensive railway data analysis with major improvements:

NEW FEATURES IN THIS VERSION:
1. Primary data source: Excel file with comprehensive infrastructure data (435 total points)
2. SWEREF99 TM to WGS84 coordinate conversion for accurate GPS matching
3. Enhanced infrastructure quality filtering and duplicate removal
4. Infrastructure density analysis along route segments
5. Improved GPS validation with infrastructure coverage scoring
6. Rich metadata preservation for enhanced downstream analysis

INFRASTRUCTURE DATA SOURCES:
- PRIMARY: Emaint_Bandel_331_Borlänge_Mora_Turnout.xlsx (comprehensive, 435 points total)
  * Turnouts: 75 points (with detailed switch information)
  * Bridges: 25 points (with structural details) 
  * RailJoints: 335 points (16x more than CSV files!)
- FALLBACK: CSV files (limited, 120 points total)

COORDINATE SYSTEM HANDLING:
- Excel data: SWEREF99 TM (Swedish national grid system)
- GPS data: WGS84 (World Geodetic System 1984)
- Automatic conversion between systems for accurate matching

OUTPUT FILES (Enhanced for Code 2):
- railway_map_static.png: Static infrastructure map
- infrastructure_points.csv: Main infrastructure data for Code 2
- valid_folders.txt: Quality-validated GPS folders
- folder_validation_report.txt: Detailed validation results
- code1_center.txt: Map center coordinates for reference

Author: Studenka Lundahl
"""

import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime

# Install: pip install pyproj
from pyproj import Transformer

# Clean output for better readability
warnings.filterwarnings("ignore")

#############################################################
# COORDINATE SYSTEM CONVERSION FUNCTIONS
#############################################################

def convert_sweref99_to_wgs84(northing, easting):
    """
    Convert SWEREF99 TM coordinates to WGS84 latitude/longitude using pyproj
    
    SWEREF99 TM is the Swedish national coordinate system used in official infrastructure databases.
    This function provides approximate conversion to WGS84 for GPS matching.
    
    For production systems, use proper projection libraries like pyproj for exact conversion.
    
    Args:
        northing: SWEREF99 TM northing coordinate (Y-axis, meters from equator)
        easting: SWEREF99 TM easting coordinate (X-axis, meters from central meridian)
    
    Returns:
        tuple: (latitude, longitude) in WGS84 decimal degrees
    """
    # Create transformer from SWEREF99 TM to WGS84
    transformer = Transformer.from_crs("EPSG:3006", "EPSG:4326", always_xy=True)
    
    # Convert coordinates (note: pyproj expects easting, northing order)
    longitude, latitude = transformer.transform(easting, northing)
    
    return latitude, longitude

def load_infrastructure_from_excel(excel_file_path):
    """
    Load comprehensive infrastructure data from the Excel database.
    
    This function extracts infrastructure data from the official railway database,
    providing significantly more complete coverage than the CSV files.
    
    MAJOR IMPROVEMENT: RailJoint coverage increases from 20 to 335 points (16x more data!)
    
    Args:
        excel_file_path: Path to the Emaint_Bandel_331_Borlänge_Mora_Turnout.xlsx file
    
    Returns:
        pandas.DataFrame: Combined infrastructure data with WGS84 coordinates
    """
    print("🎯 LOADING INFRASTRUCTURE DATA FROM EXCEL")
    print("="*60)
    
    excel_data = []
    
    # Define Excel sheets and their corresponding infrastructure categories
    sheets_to_read = {
        'Resultat_Turnout': 'Turnout',     # Railway switches and junctions
        'Resultat_Bridge': 'Bridge',       # Railway bridges and overpasses  
        'Resultat_RailJoint': 'RailJoint'  # Rail joints and connections
    }
    
    for sheet_name, category in sheets_to_read.items():
        try:
            print(f"📊 Processing {sheet_name} for {category} infrastructure...")
            
            # Read the Excel sheet
            df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            print(f"   • Loaded {len(df)} rows from {sheet_name}")
            
            # Look for coordinate columns (different naming in different sheets)
            coord_columns = {
                'northing': None,
                'easting': None
            }
            
            # Find coordinate columns (they have different names in different sheets)
            for col in df.columns:
                col_lower = str(col).lower()
                if 'northing' in col_lower and coord_columns['northing'] is None:
                    coord_columns['northing'] = col
                elif 'easting' in col_lower and coord_columns['easting'] is None:
                    coord_columns['easting'] = col
            
            if coord_columns['northing'] and coord_columns['easting']:
                print(f"   • Found coordinates: {coord_columns['northing']}, {coord_columns['easting']}")
                
                # Extract valid coordinate pairs
                valid_rows = df[
                    pd.notna(df[coord_columns['northing']]) & 
                    pd.notna(df[coord_columns['easting']])
                ]
                
                if len(valid_rows) > 0:
                    # Convert SWEREF99 TM coordinates to WGS84
                    print(f"   • Converting {len(valid_rows)} coordinate pairs from SWEREF99 TM to WGS84...")
                    
                    latitudes = []
                    longitudes = []
                    
                    for _, row in valid_rows.iterrows():
                        northing = float(row[coord_columns['northing']])
                        easting = float(row[coord_columns['easting']])
                        
                        # Convert coordinates
                        lat, lon = convert_sweref99_to_wgs84(northing, easting)
                        latitudes.append(lat)
                        longitudes.append(lon)
                    
                    # Create result DataFrame
                    result_df = pd.DataFrame({
                        'Latitude': latitudes,
                        'Longitude': longitudes,
                        'Category': category,
                        'Source': 'excel_comprehensive',
                        'Original_Northing': valid_rows[coord_columns['northing']].values,
                        'Original_Easting': valid_rows[coord_columns['easting']].values
                    })
                    
                    # Basic coordinate validation (should be in Sweden)
                    valid_coords = (
                        (result_df['Latitude'] >= 55.0) & (result_df['Latitude'] <= 70.0) &
                        (result_df['Longitude'] >= 10.0) & (result_df['Longitude'] <= 25.0)
                    )
                    
                    result_df = result_df[valid_coords].reset_index(drop=True)
                    
                    if len(result_df) > 0:
                        excel_data.append(result_df)
                        print(f"   ✅ Successfully converted {len(result_df)} {category} points")
                        print(f"   📍 Latitude between {result_df['Latitude'].min():.6f}° and {result_df['Latitude'].max():.6f}°")
                        print(f"   📍 Longitude between {result_df['Longitude'].min():.6f}° and {result_df['Longitude'].max():.6f}°")

                    else:
                        print(f"   ⚠️ No valid coordinates after conversion for {category}")
                else:
                    print(f"   ⚠️ No valid coordinate data found in {sheet_name}")
            else:
                print(f"   ❌ Coordinate columns not found in {sheet_name}")
                print(f"   Available columns: {list(df.columns)}")
                
        except Exception as e:
            print(f"   ❌ Error processing {sheet_name}: {e}")
    
    # Combine all infrastructure data
    if excel_data:
        combined_data = pd.concat(excel_data, ignore_index=True)
        print(f"\n✅ EXCEL DATA INTEGRATION COMPLETE:")
        print(f"   • Total infrastructure points: {len(combined_data)}")
        
        # Show category distribution
        category_counts = combined_data['Category'].value_counts()
        for category, count in category_counts.items():
            print(f"   • {category}: {count} points")
        
        return combined_data
    else:
        print(f"\n❌ No valid data extracted from Excel file")
        return pd.DataFrame()

def load_infrastructure_from_csv_fallback():
    """
    Fallback function to load infrastructure data from CSV files.
    
    This function provides the original CSV-based loading as a backup
    in case the Excel file is not available or cannot be processed.
    
    Returns:
        pandas.DataFrame: Infrastructure data from CSV files
    """
    print("⚠️ FALLBACK: LOADING LIMITED DATA FROM CSV FILES")
    print("="*50)
    
    base_path = os.path.join(".", "Data 1")
    
    files = {
        "Bridge": os.path.join(base_path, "converted_coordinates_Resultat_Bridge.csv"),
        "RailJoint": os.path.join(base_path, "converted_coordinates_Resultat_RailJoint.csv"),
        "Turnout": os.path.join(base_path, "converted_coordinates_Turnout.csv")   
    }
    
    # Initialize an empty list to hold DataFrames
    data_frames = []
    
    # Loop through each infrastructure category and load the corresponding CSV file
    for category, file in files.items():
        try:
            print(f"📂 Loading {category} from {os.path.basename(file)}...")
            # Load the CSV file into a DataFrame with UTF-8 encoding
            df = pd.read_csv(file, encoding="utf-8")
            
            # Clean up column names and ensure Latitude and Longitude are numeric
            if not df.empty and "Latitude" in df.columns and "Longitude" in df.columns:
                df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
                df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
                df = df[["Latitude", "Longitude"]].dropna()
                df["Category"] = category   # Add category column for identification
                df["Source"] = "csv_limited"
                data_frames.append(df)
                print(f"   ✅ Loaded {len(df)} {category} points")
            else:
                print(f"   ❌ Invalid or empty file: {file}")
                
        except Exception as e:
            print(f"   ❌ Error loading {category}: {e}")
    
    # Combine all infrastructure data into single DataFrame
    if data_frames:
        combined_data = pd.concat(data_frames, ignore_index=True)
        print(f"\n📊 CSV fallback data loaded: {len(combined_data)} total points")
        return combined_data
    else:
        print(f"\n❌ No valid CSV data found")
        return pd.DataFrame()

#############################################################
# INFRASTRUCTURE QUALITY ANALYSIS AND FILTERING
#############################################################

def analyze_infrastructure_density(data, route_segments=5):
    """
    Analyze infrastructure distribution along the railway route.
    
    This analysis helps understand how infrastructure is distributed
    geographically, which is important for balanced model training.
    
    Args:
        data: DataFrame containing infrastructure points
        route_segments: Number of route segments to analyze
    """
    print(f"\n📊 INFRASTRUCTURE DENSITY ANALYSIS:")
    print("="*50)
    
    if data.empty:
        print("No infrastructure data to analyze")
        return
    
    # Divide route into segments based on latitude (north-south direction)
    lat_min, lat_max = data['Latitude'].min(), data['Latitude'].max()
    lat_segments = np.linspace(lat_min, lat_max, route_segments + 1)
    
    print(f"Route analysis: {lat_min:.3f}°N to {lat_max:.3f}°N divided into {route_segments} segments")
    
    total_infrastructure = len(data)
    
    for i in range(route_segments):
        # Define segment boundaries
        segment_start = lat_segments[i]
        segment_end = lat_segments[i + 1]
        
        # Filter data for this segment
        segment_data = data[
            (data['Latitude'] >= segment_start) & 
            (data['Latitude'] < segment_end)
        ]
        
        if not segment_data.empty:
            segment_stats = segment_data['Category'].value_counts()
            segment_percentage = len(segment_data) / total_infrastructure * 100
            
            print(f"\n📍 Route Segment {i+1}:")
            print(f"   • Latitude range: {segment_start:.3f}°N - {segment_end:.3f}°N")
            print(f"   • Total infrastructure: {len(segment_data)} ({segment_percentage:.1f}% of route)")
            
            for category, count in segment_stats.items():
                category_percentage = count / len(segment_data) * 100
                print(f"   • {category}: {count} points ({category_percentage:.1f}%)")
        else:
            print(f"\n📍 Route Segment {i+1}: No infrastructure (sparse area)")

def filter_infrastructure_quality(data, min_distance_between_same_type=0.0008):
    """
    Remove duplicate or very close infrastructure points to improve data quality.
    
    Infrastructure databases sometimes contain duplicate entries or points
    that are extremely close together, which can cause over-labeling in Code 2.
    
    Args:
        data: DataFrame containing infrastructure points
        min_distance_between_same_type: Minimum distance (degrees) between same-type infrastructure
    
    Returns:
        pandas.DataFrame: Filtered infrastructure data
    """
    print(f"\n🔧 INFRASTRUCTURE QUALITY FILTERING:")
    print("="*40)
    
    if data.empty:
        return data
    
    filtered_data = []
    min_distance_meters = min_distance_between_same_type * 111000  # Convert to meters for display
    
    print(f"Removing infrastructure points closer than {min_distance_meters:.0f}m of same type...")
    
    for category in data['Category'].unique():
        category_data = data[data['Category'] == category].copy().reset_index(drop=True)
        
        if len(category_data) <= 1:
            filtered_data.append(category_data)
            print(f"   • {category}: {len(category_data)} points (no filtering needed)")
            continue
        
        # Filter out points that are too close to each other
        to_keep = [0]  # Always keep the first point
        
        for i in range(1, len(category_data)):
            current_lat = category_data.iloc[i]['Latitude']
            current_lon = category_data.iloc[i]['Longitude']
            
            # Check distance to all previously kept points
            keep_point = True
            for kept_idx in to_keep:
                kept_lat = category_data.iloc[kept_idx]['Latitude']
                kept_lon = category_data.iloc[kept_idx]['Longitude']
                
                # Calculate approximate distance in degrees
                lat_diff = current_lat - kept_lat
                lon_diff = (current_lon - kept_lon) * np.cos(np.radians(current_lat))
                distance = np.sqrt(lat_diff**2 + lon_diff**2)
                
                if distance < min_distance_between_same_type:
                    keep_point = False
                    break
            
            if keep_point:
                to_keep.append(i)
        
        # Create filtered dataset for this category
        filtered_category = category_data.iloc[to_keep].reset_index(drop=True)
        filtered_data.append(filtered_category)
        
        removed_count = len(category_data) - len(filtered_category)
        print(f"   • {category}: {len(category_data)} → {len(filtered_category)} points (removed {removed_count} duplicates)")
    
    # Combine filtered data
    result = pd.concat(filtered_data, ignore_index=True)
    total_removed = len(data) - len(result)
    print(f"\n📊 Quality filtering complete:")
    print(f"   • Original points: {len(data)}")
    print(f"   • Filtered points: {len(result)}")
    print(f"   • Removed duplicates: {total_removed}")
    
    return result

#############################################################
# GPS VALIDATION WITH INFRASTRUCTURE SCORING
#############################################################

def enhanced_gps_validation(lat_df, lon_df, speed_df, satellites_df, infrastructure_coords):
    """
    GPS validation that includes infrastructure coverage assessment.
    
    This validation adds infrastructure matching to the quality criteria,
    ensuring that validated folders actually contain relevant railway infrastructure.
    
    Args:
        lat_df, lon_df, speed_df, satellites_df: GPS data DataFrames
        infrastructure_coords: DataFrame containing infrastructure coordinates
    
    Returns:
        tuple: (is_valid, issues_list, satellite_info, infrastructure_stats)
    """
    # Perform basic GPS quality validation
    is_valid, issues, satellites_info = validate_gps_folder_quality(
        lat_df, lon_df, speed_df, satellites_df
    )
    
    infrastructure_stats = {
        'matches': 0,
        'coverage_percentage': 0,
        'infrastructure_types': [],
        'quality_score': 0
    }
    
    # Only proceed with infrastructure analysis if basic quality passes
    if is_valid and not infrastructure_coords.empty:
        try:
            # Convert GPS data to numeric
            lats = pd.to_numeric(lat_df.iloc[:, 0], errors='coerce').dropna()
            lons = pd.to_numeric(lon_df.iloc[:, 0], errors='coerce').dropna()
            
            if len(lats) > 0 and len(lons) > 0:
                # Count infrastructure matches within 500m
                infrastructure_matches = 0
                infrastructure_types = set()
                
                for _, infra_row in infrastructure_coords.iterrows():
                    infra_lat, infra_lon = infra_row['Latitude'], infra_row['Longitude']
                    
                    # Calculate distances in meters (approximate)
                    lat_diff = (lats - infra_lat) * 111000  # 1 degree lat ≈ 111km
                    lon_diff = (lons - infra_lon) * 111000 * np.cos(np.radians(infra_lat))
                    distances = np.sqrt(lat_diff**2 + lon_diff**2)
                    
                    # Check if any GPS point comes within 500m
                    if np.any(distances < 500):
                        infrastructure_matches += 1
                        infrastructure_types.add(infra_row['Category'])
                
                # Calculate coverage metrics
                total_infrastructure = len(infrastructure_coords)
                coverage_percentage = (infrastructure_matches / total_infrastructure) * 100 if total_infrastructure > 0 else 0
                diversity_score = len(infrastructure_types)
                
                # Update infrastructure statistics
                infrastructure_stats.update({
                    'matches': infrastructure_matches,
                    'coverage_percentage': coverage_percentage,
                    'infrastructure_types': list(infrastructure_types),
                    'quality_score': coverage_percentage + (diversity_score * 10)  # Bonus for type diversity
                })
                
                # Apply enhanced validation criteria
                if coverage_percentage < 15:  # Require at least 15% infrastructure coverage
                    issues.append(f"Low infrastructure coverage: {coverage_percentage:.1f}% (minimum 15% required)")
                    is_valid = False
                
                if len(infrastructure_types) < 2:  # Require at least 2 different infrastructure types
                    issues.append(f"Insufficient infrastructure diversity: {list(infrastructure_types)} (minimum 2 types required)")
                    is_valid = False
                    
        except Exception as e:
            issues.append(f"Infrastructure analysis error: {str(e)}")
            is_valid = False
    
    return is_valid, issues, satellites_info, infrastructure_stats

#############################################################
# GPS Track Quality Validation for Noise Filtering
#############################################################

def validate_gps_folder_quality(lat_df, lon_df, speed_df, satellites_df=None):
    """
    Validates GPS data quality to filter out folders with tracking issues.
    
    This function checks for:
    - Sufficient data points for meaningful analysis
    - GPS coordinates within the Mora-Borlänge region
    - Realistic train speeds and movement patterns
    - Track continuity without excessive position jumps
    - GPS satellite signal quality when available
    
    Args:
        lat_df: DataFrame containing latitude data
        lon_df: DataFrame containing longitude data  
        speed_df: DataFrame containing speed data
        satellites_df: Optional DataFrame containing GPS satellite count data
    
    Returns:
        tuple: (is_valid_boolean, list_of_issues, satellite_info_string)
    """
    issues = []
    
    # Convert GPS data to numeric format and remove invalid values
    lats = pd.to_numeric(lat_df.iloc[:, 0], errors='coerce').dropna()
    lons = pd.to_numeric(lon_df.iloc[:, 0], errors='coerce').dropna()
    speeds = pd.to_numeric(speed_df.iloc[:, 0], errors='coerce').dropna()
    
    # Define Mora-Borlänge route boundaries
    MORA_BORLANGE_LAT_RANGE = (60.4, 61.2)  # Latitude range covering the route
    MORA_BORLANGE_LON_RANGE = (14.0, 15.5)  # Longitude range covering the route
    
    # Check for sufficient data points
    if len(lats) < 1000:
        issues.append(f"Insufficient GPS data: {len(lats)} points (minimum 1000 required)")
    
    # Verify GPS coordinates are within the Mora-Borlänge railway region
    if not (MORA_BORLANGE_LAT_RANGE[0] <= lats.min() and lats.max() <= MORA_BORLANGE_LAT_RANGE[1]):
        issues.append("GPS coordinates outside Mora-Borlänge latitude range")
    
    if not (MORA_BORLANGE_LON_RANGE[0] <= lons.min() and lons.max() <= MORA_BORLANGE_LON_RANGE[1]):
        issues.append("GPS coordinates outside Mora-Borlänge longitude range")
    
    # Check for realistic train speeds
    if len(speeds) > 0 and (speeds.max() > 180 or speeds.min() < -10):
        issues.append(f"Unrealistic train speeds: {speeds.min():.1f} to {speeds.max():.1f} km/h")
    
    # Validate track continuity by checking for unrealistic position jumps
    if len(lats) > 10:
        # Calculate distances between consecutive GPS points to detect tracking errors
        distances_km = []
        for i in range(1, min(len(lats), 1000)):    # Sample first 1000 points for performance
            lat_diff = (lats.iloc[i] - lats.iloc[i-1]) * 111.0  # 1 degree latitude ≈ 111 km
            lon_diff = (lons.iloc[i] - lons.iloc[i-1]) * 111.0 * np.cos(np.radians(lats.iloc[i]))
            distance = np.sqrt(lat_diff**2 + lon_diff**2)
            distances_km.append(distance)
        
        if distances_km:
            max_jump = max(distances_km)
            if max_jump > 5.0:
                issues.append(f"Unrealistic position jump: {max_jump:.1f} km between consecutive GPS points")
            
            # Check if track is too fragmented with many large gaps
            big_jumps = sum(1 for d in distances_km if d > 1.0)  # Count jumps > 1km
            if big_jumps > len(distances_km) * 0.15:  # More than 15% large jumps indicates problems
                issues.append(f"Track too fragmented: {big_jumps}/{len(distances_km)} position jumps >1km")
    
    # Validate speed consistency to detect acceleration/deceleration anomalies
    if len(speeds) > 10:
        # Calculate speed changes between consecutive measurements
        speed_changes = np.abs(np.diff(speeds[:min(len(speeds), 1000)]))  # Sample for performance
        
        # Check for unrealistic acceleration or deceleration
        extreme_changes = sum(1 for change in speed_changes if change > 50)  # >50 km/h change is extreme
        if extreme_changes > len(speed_changes) * 0.08:  # More than 8% extreme changes indicates issues
            issues.append(f"Unrealistic speed changes: {extreme_changes}/{len(speed_changes)} changes >50 km/h")
    
    # Validate route length to ensure meaningful track segments
    if len(lats) > 1:
        # Calculate approximate total track length
        total_distance = 0
        for i in range(1, min(len(lats), 2000)):  # Sample for performance
            lat_diff = (lats.iloc[i] - lats.iloc[i-1]) * 111.0
            lon_diff = (lons.iloc[i] - lons.iloc[i-1]) * 111.0 * np.cos(np.radians(lats.iloc[i]))
            total_distance += np.sqrt(lat_diff**2 + lon_diff**2)
        
        # Scale up if we only sampled part of the data
        if len(lats) > 2000:
            total_distance = total_distance * (len(lats) / 2000)
            
        if total_distance < 6.0:  # Routes should be substantial for analysis
            issues.append(f"Track segment too short: {total_distance:.1f} km (minimum 6km required)")
        
        if total_distance > 300.0:  # Extremely long routes may include noise
            issues.append(f"Track suspiciously long: {total_distance:.1f} km (may include GPS noise)")

    # Validate movement patterns to ensure actual train operation
    if len(speeds) > 50:
         # Check if the train actually moves during the recording
        moving_points = sum(1 for s in speeds if s > 8.0)   # Points where speed > 8 km/h
        moving_percentage = (moving_points / len(speeds)) * 100
        
        if moving_percentage < 25:  # Less than 25% moving points suggests stationary recording
            issues.append(f"Mostly stationary recording: only {moving_percentage:.1f}% of points show movement")
    
    # Assess GPS satellite signal quality when available
    satellites_info = "No satellite data available"
    if satellites_df is not None and not satellites_df.empty:
        try:
            sats = pd.to_numeric(satellites_df.iloc[:, 0], errors='coerce').dropna()
            if len(sats) > 0:
                avg_sats = sats.mean()
                poor_gps_percent = (sats < 4).sum() / len(sats) * 100
                very_poor_gps_percent = (sats < 3).sum() / len(sats) * 100
                satellites_info = f"Average satellites: {avg_sats:.1f}, Poor GPS (<4 sats): {poor_gps_percent:.1f}%, Very poor (<3 sats): {very_poor_gps_percent:.1f}%"
                
                # Apply stricter GPS quality requirements for reliable tracking
                if poor_gps_percent > 45:   # Reduced tolerance for poor GPS
                    issues.append(f"Excessive poor GPS quality: {poor_gps_percent:.1f}% with <4 satellites")
                
                if very_poor_gps_percent > 20:
                    issues.append(f"Excessive very poor GPS: {very_poor_gps_percent:.1f}% with <3 satellites")
        except Exception as e:
            satellites_info = f"Satellite data processing error: {str(e)}"

    # Return validation result 
    is_valid = len(issues) == 0
    return is_valid, issues, satellites_info

#############################################################
# MAIN EXECUTION: INFRASTRUCTURE LOADING
#############################################################

print("🚀 RAILWAY INFRASTRUCTURE ANALYSIS")
print("="*60)
print(f"Execution time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Define file paths
base_path = os.path.join(".", "Data 1")
excel_file = os.path.join(base_path, "Emaint_Bandel_331_Borlänge_Mora_Turnout.xlsx")

# Define marker styles with different colors and sizes for each infrastructure type
marker_styles = {
    "Bridge": {"color": "red", "size": 10},
    "RailJoint": {"color": "blue", "size": 8},
    "Turnout": {"color": "green", "size": 12}
}

# Primary: Load from Excel file (comprehensive data)
data = pd.DataFrame()

if os.path.exists(excel_file):
    print(f"📁 Primary source found: {os.path.basename(excel_file)}")
    data = load_infrastructure_from_excel(excel_file)
    
    if data.empty:
        print("❌ Excel file exists but no valid data extracted. Trying CSV fallback...")
        data = load_infrastructure_from_csv_fallback()
else:
    print(f"⚠️ Excel file not found: {excel_file}")
    print("Using CSV files as data source...")
    data = load_infrastructure_from_csv_fallback()

# Validate that we have infrastructure data
if data.empty:
    print("❌ CRITICAL ERROR: No infrastructure data loaded from any source!")
    print("Please check that either the Excel file or CSV files exist in the Data 1 folder.")
    exit(1)

print(f"\n✅ INFRASTRUCTURE DATA LOADED SUCCESSFULLY:")
print(f"   • Total infrastructure points: {len(data)}")
print(f"   • Data source: {data['Source'].iloc[0] if 'Source' in data.columns else 'unknown'}")

# Show category distribution
category_counts = data['Category'].value_counts()
print(f"\n📊 Infrastructure distribution:")
for category, count in category_counts.items():
    percentage = count / len(data) * 100
    print(f"   • {category}: {count} points ({percentage:.1f}%)")

# Show coordinate ranges
print(f"\n📍 Geographic coverage:")
print(f"   • Latitude range: {data['Latitude'].min():.6f}° to {data['Latitude'].max():.6f}°")
print(f"   • Longitude range: {data['Longitude'].min():.6f}° to {data['Longitude'].max():.6f}°")

# Perform infrastructure analysis and quality filtering
analyze_infrastructure_density(data, route_segments=5)
data = filter_infrastructure_quality(data)

# Display final statistics
print(f"\n📊 FINAL INFRASTRUCTURE STATISTICS:")
final_counts = data['Category'].value_counts()
for category, count in final_counts.items():
    print(f"   • {category}: {count} points (ready for Code 2)")

#############################################################
# VISUALIZATION: INFRASTRUCTURE MAP
#############################################################

print(f"\n🗺️ CREATING INFRASTRUCTURE VISUALIZATION:")

# Create interactive map
fig = go.Figure()

# Add infrastructure points by category
for category, style in marker_styles.items():
    category_data = data[data["Category"] == category]
    
    if not category_data.empty:
        fig.add_trace(go.Scattermap(
            lat=category_data["Latitude"],
            lon=category_data["Longitude"],
            mode="markers",
            marker=dict(
                color=style["color"],
                size=style["size"],
                opacity=0.8
            ),
            name=f"{category} ({len(category_data)})",
            hovertemplate=f"<b>{category}</b><br>" +
                         "Lat: %{lat:.6f}<br>" +
                         "Lon: %{lon:.6f}<br>" +
                         "<extra></extra>"
        ))
        print(f"   • Added {len(category_data)} {category} points to map")

# Enhanced map layout
fig.update_layout(
    title={
        'text': f"Railway Map with Bridges, RailJoints and Turnouts<br>Total: {len(data)} points from {data['Source'].iloc[0] if 'Source' in data.columns else 'mixed'} source(s)",
        'x': 0.5,
        'xanchor': 'center'
    },
    map_style="open-street-map",    # Use OpenStreetMap for detailed railway visualization
    legend_title_text="Infrastructure Types",
    width=1400,
    height=900,
    margin={"r": 0, "t": 80, "l": 50, "b": 0},  # Set margins for the figure
    map=dict(
        zoom=9,
        center=dict(
            lat=data["Latitude"].mean(), 
            lon=data["Longitude"].mean()
        )
    )
)

#############################################################
# INTERACTIVE MAP DISPLAY AND EXPORT
#############################################################
"""
This section displays the interactive railway infrastructure map and exports it
for Grade 3 submission requirements and documentation purposes.

The map visualization provides:
- Interactive exploration of infrastructure locations
- Professional cartographic representation using OpenStreetMap
- Color-coded infrastructure types for clear identification
- Zoom, pan, and hover functionality for detailed analysis
"""

# Display interactive map
print("   • Displaying interactive map...")
fig.show()

#############################################################
# RAILWAY MAP EXPORT FOR DOCUMENTATION
#############################################################
"""
Export the railway infrastructure map for documentation and analysis purposes.

HTML format is chosen over PNG for several technical and practical advantages:
1. RELIABILITY: Always works regardless of kaleido/system configuration issues
2. INTERACTIVITY: Preserves zoom, pan, and hover functionality for demonstration
3. QUALITY: Vector-based graphics that scale perfectly at any resolution
4. COMPATIBILITY: Opens in any web browser on any operating system
5. PROFESSIONAL: Self-contained file with embedded JavaScript and styling
6. PRESERVATION: Maintains all interactive features for detailed infrastructure analysis

The exported HTML file serves as both a static visualization and an interactive
demonstration tool for presenting railway infrastructure analysis results.
"""

print("📷 Saving railway map for documentation and analysis...")

# Export interactive HTML map with embedded dependencies
fig.write_html(
    "railway_map_static.html",          # Output filename for documentation
    include_plotlyjs=True,              # Embed Plotly.js for offline viewing
    config={
        'displayModeBar': True,         # Show zoom/pan toolbar
        'displaylogo': False,           # Remove Plotly logo for cleaner presentation
        'modeBarButtonsToRemove': [     # Remove unnecessary toolbar buttons
            'lasso2d', 'select2d'
        ]
    }
)

print("💾 Interactive railway map saved: railway_map_static.html")
print("📄 Open railway_map_static.html in any web browser to view the complete infrastructure analysis")
print("🎯 HTML format provides superior interactive capabilities for railway infrastructure visualization")
    
#############################################################
# GPS VALIDATION WITH INFRASTRUCTURE MATCHING
#############################################################

# Define path to GPS measurement data folders
data2_root = os.path.join(".", "Data 2")

# Check if the Data 2 folder exists
if not os.path.exists(data2_root):
    print(f"⚠️ Data 2 folder not found. Skipping GPS validation.")
    valid_folders = []
    folder_stats = {}
else:
    print(f"\n🔍 GPS VALIDATION WITH INFRASTRUCTURE MATCHING:")
    print("="*60)
    
    # Initialize storage for validation results
    valid_folders = []  # Folders that pass both quality and infrastructure checks
    folder_stats = {}   # Detailed statistics for each folder
    
    print("Starting folder validation...")

    # Scan folders in Data 2 directory
    for folder in os.listdir(data2_root):
        # Skip non-directory items and hidden folders
        if not os.path.isdir(os.path.join(data2_root, folder)) or folder.startswith('.'):
            continue
        
        print(f"\nChecking folder: {folder}")
        base_path = os.path.join(data2_root, folder)
        
        # Handle both flat and nested folder structures
        possible_paths = [base_path, os.path.join(base_path, folder)]
        possible_paths = [p for p in possible_paths if os.path.isdir(p)]
        
        print(f"📂 Analyzing folder: {folder}")
        
        if not possible_paths:
            print(f"    ⚠️ No valid directory structure found. Skipping.")
            continue
        
        # Check each possible location for GPS data files
        found = False
        for subfolder in possible_paths:
            # Define required GPS data files
            lat_file = os.path.join(subfolder, "GPS.latitude.csv")
            lon_file = os.path.join(subfolder, "GPS.longitude.csv")
            speed_file = os.path.join(subfolder, "GPS.speed.csv")
            satellites_file = os.path.join(subfolder, "GPS.satellites.csv")
            
            # Check if essential GPS files exist
            if os.path.isfile(lat_file) and os.path.isfile(lon_file):
                found = True
                try:
                    # Load GPS coordinate files
                    lat_df = pd.read_csv(lat_file, header=None)
                    lon_df = pd.read_csv(lon_file, header=None)
                    
                    # Load speed data file if available
                    speed_df = None
                    if os.path.isfile(speed_file):
                        speed_df = pd.read_csv(speed_file, header=None)
                    
                    # Load satellite data file if available
                    satellites_df = None
                    if os.path.isfile(satellites_file):
                        try:
                            satellites_df = pd.read_csv(satellites_file, header=None)
                            print(f"    📡 GPS satellite data found for quality assessment")
                        except Exception:
                            print(f"    ⚠️ Satellite file exists but couldn't load")
                    
                    # Check for empty GPS files
                    if lat_df.empty or lon_df.empty:
                        print(f"    ❌ Empty GPS files detected. Skipping folder.")
                        break
                    
                    # Perform infrastructure-aware quality validation if speed data is available
                    if speed_df is not None and not speed_df.empty:
                        quality_valid, quality_issues, satellite_info, infrastructure_stats = enhanced_gps_validation(
                            lat_df, lon_df, speed_df, satellites_df, data
                        )
                        
                        print(f"    📊 GPS satellite info: {satellite_info}")
                        print(f"    🏗️ Infrastructure coverage: {infrastructure_stats['coverage_percentage']:.1f}%")
                        print(f"    🎯 Infrastructure types found: {infrastructure_stats['infrastructure_types']}")
                        
                        # Check if folder passes quality validation
                        if not quality_valid:
                            print(f"    ❌ Enhanced GPS validation failed:")
                            for issue in quality_issues:
                                print(f"        • {issue}")
                            
                            # Store detailed failure information
                            folder_stats[folder] = {
                                "validation_status": "failed",
                                "quality_issues": quality_issues,
                                "satellites_info": satellite_info,
                                "infrastructure_stats": infrastructure_stats
                            }
                            break
                        else:
                            # Folder passes enhanced validation
                            valid_folders.append(folder)
                            folder_stats[folder] = {
                                "validation_status": "passed",
                                "satellites_info": satellite_info,
                                "infrastructure_stats": infrastructure_stats,
                                "quality_score": infrastructure_stats['quality_score']
                            }
                            
                            print(f"    ✅ FOLDER VALIDATED - Enhanced criteria passed!")
                            print(f"        • Infrastructure matches: {infrastructure_stats['matches']} points")
                            print(f"        • Infrastructure types: {', '.join(infrastructure_stats['infrastructure_types'])}")
                            print(f"        • Route coverage: {infrastructure_stats['coverage_percentage']:.1f}%")
                            print(f"        • Quality score: {infrastructure_stats['quality_score']:.1f}")
                    else:
                        print(f"    ❌ No speed data available for validation")
                        
                except Exception as e:
                    print(f"    ❌ Error processing GPS files: {e}")
                break
        
        if not found:
            print(f"    ⚠️ No GPS data files found in folder")
    
    # Display validation results summary
    print(f"\n✅ FOLDER VALIDATION COMPLETED!")
    print(f"Valid folders found: {len(valid_folders)}")
    
    if valid_folders:
        print(f"\n📊 VALIDATION SUMMARY:")
        total_folders = len([f for f in os.listdir(data2_root) 
                           if not f.startswith('.') and os.path.isdir(os.path.join(data2_root, f))])
        print(f"   • Total folders scanned: {total_folders}")
        print(f"   • Valid folders identified: {len(valid_folders)}")
        print(f"   • Success rate: {(len(valid_folders)/total_folders*100):.1f}%")
        
        print(f"\n📋 Valid folders for Code 2 analysis:")
        # Sort by quality score (highest first)
        sorted_folders = sorted(valid_folders, 
                              key=lambda f: folder_stats[f]['quality_score'], 
                              reverse=True)
        
        for folder in sorted_folders:
            stats = folder_stats[folder]
            infra_stats = stats['infrastructure_stats']
            types_str = ", ".join(infra_stats['infrastructure_types'])
            print(f"   • {folder} (Score: {infra_stats['quality_score']:.1f}, Types: {types_str})")

#############################################################
# OUTPUT FILES FOR CODE 2
#############################################################

print(f"\n💾 CREATING OUTPUT FILES FOR CODE 2:")
print("="*50)

# Main infrastructure file for use by Code 2
main_output = data[['Latitude', 'Longitude', 'Category']].copy()
main_output.to_csv("infrastructure_points.csv", index=False)
print(f"💾 Saved: infrastructure_points.csv ({len(main_output)} points)")
#print(f"✅ Primary output: infrastructure_points.csv ({len(main_output)} points)")


# Valid folders list for Code 2
if 'valid_folders' in locals() and valid_folders:
    with open("valid_folders.txt", "w") as f:
        for folder in valid_folders:
            f.write(folder + "\n")
    print(f"💾 Saved: valid_folders.txt ({len(valid_folders)} folders)")
else:
    print(f"⚠️ No valid folders to export")

# Save map center coordinates for reference
with open("code1_center.txt", "w") as f:
    f.write(f"{data['Latitude'].mean():.6f},{data['Longitude'].mean():.6f}")
print(f"💾 Saved: code1_center.txt")

# Comprehensive validation report
if 'folder_stats' in locals():
    with open("folder_validation_report.txt", "w") as f:
        f.write("MORA-BORLÄNGE RAILWAY GPS VALIDATION REPORT\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Infrastructure data source: {data['Source'].iloc[0] if 'Source' in data.columns else 'mixed'}\n")
        f.write(f"Total infrastructure points: {len(data)}\n\n")
        
        f.write("INFRASTRUCTURE DISTRIBUTION:\n")
        f.write("-" * 30 + "\n")
        for category, count in data['Category'].value_counts().items():
            percentage = count / len(data) * 100
            f.write(f"• {category}: {count} points ({percentage:.1f}%)\n")
        
        f.write(f"\n\nVALIDATED FOLDERS (Enhanced Criteria):\n")
        f.write("-" * 40 + "\n")
        if valid_folders:
            for folder in valid_folders:
                stats = folder_stats.get(folder, {})
                infra_stats = stats.get('infrastructure_stats', {})
                f.write(f"\n📁 {folder}:\n")
                f.write(f"   - Infrastructure coverage: {infra_stats.get('coverage_percentage', 0):.1f}%\n")
                f.write(f"   - Infrastructure types: {', '.join(infra_stats.get('infrastructure_types', []))}\n")
                f.write(f"   - Infrastructure matches: {infra_stats.get('matches', 0)}\n")
                f.write(f"   - Quality score: {infra_stats.get('quality_score', 0):.1f}\n")
                f.write(f"   - Validation status: PASSED\n")
        else:
            f.write("No folders passed enhanced validation criteria.\n")
        
        f.write(f"\n\nREJECTED FOLDERS:\n")
        f.write("-" * 20 + "\n")
        rejected_folders = [folder for folder, stats in folder_stats.items() 
                          if stats.get('validation_status') == 'failed']
        
        if rejected_folders:
            for folder in rejected_folders:
                stats = folder_stats[folder]
                f.write(f"\n📁 {folder}:\n")
                f.write(f"   - Validation status: FAILED\n")
                if 'quality_issues' in stats:
                    f.write(f"   - Issues found:\n")
                    for issue in stats['quality_issues']:
                        f.write(f"     • {issue}\n")
                infra_stats = stats.get('infrastructure_stats', {})
                f.write(f"   - Infrastructure coverage: {infra_stats.get('coverage_percentage', 0):.1f}%\n")
        else:
            f.write("All scanned folders either passed validation or had no GPS data.\n")
        
        f.write(f"\n\nSUMMARY STATISTICS:\n")
        f.write("-" * 20 + "\n")
        total_folders = len(folder_stats)
        f.write(f"- Total folders scanned: {total_folders}\n")
        f.write(f"- Valid folders: {len(valid_folders)}\n")
        f.write(f"- Rejected folders: {len(rejected_folders) if rejected_folders else 0}\n")
        f.write(f"- Success rate: {(len(valid_folders) / total_folders * 100):.1f}%\n")
        f.write(f"\nENHANCED VALIDATION CRITERIA:\n")
        f.write(f"- Infrastructure coverage: minimum 15%\n")
        f.write(f"- Infrastructure diversity: minimum 2 types\n")
        f.write(f"- GPS quality: standard railway operation criteria\n")
        f.write(f"- Coordinate system: WGS84 decimal degrees\n")
    
    print("💾 Saved: folder_validation_report.txt")

#############################################################
# FINAL SUMMARY AND RECOMMENDATIONS
#############################################################

print(f"\n🎯 ENHANCED CODE 1 EXECUTION COMPLETE!")
print("="*60)

print(f"\n📊 INFRASTRUCTURE DATA SUMMARY:")
print(f"   • Data source: {data['Source'].iloc[0] if 'Source' in data.columns else 'mixed'}")
print(f"   • Total infrastructure points: {len(data)}")
for category, count in data['Category'].value_counts().items():
    print(f"   • {category}: {count} points")

if 'valid_folders' in locals():
    print(f"\n📁 GPS VALIDATION SUMMARY:")
    print(f"   • Folders validated with enhanced criteria")
    print(f"   • Valid folders for Code 2: {len(valid_folders)}")
    
    if valid_folders:
        print(f"   • Infrastructure coverage scoring applied")
        print(f"   • Quality-ranked folder list exported")

print(f"\n🚀 IMPROVEMENTS FOR CODE 2:")
if 'RailJoint' in data['Category'].values:
    railjoints = len(data[data['Category'] == 'RailJoint'])
    if railjoints > 100:
        print(f"   ✅ MAJOR IMPROVEMENT: {railjoints} RailJoint points (vs ~20 in csv-files)")
        print(f"   ✅ Expected 10-20x more RailJoint segments in Code 2")
    else:
        print(f"   📈 Moderate improvement: {railjoints} RailJoint points")
else:
    print(f"   ⚠️ No RailJoint data found - check Excel file processing")

print(f"   ✅ Enhanced coordinate accuracy from SWEREF99 TM conversion")
print(f"   ✅ Quality-filtered infrastructure points (duplicates removed)")
print(f"   ✅ Infrastructure-aware GPS folder validation")

🚀 RAILWAY INFRASTRUCTURE ANALYSIS
Execution time: 2025-08-26 15:29:47
📁 Primary source found: Emaint_Bandel_331_Borlänge_Mora_Turnout.xlsx
🎯 LOADING INFRASTRUCTURE DATA FROM EXCEL
📊 Processing Resultat_Turnout for Turnout infrastructure...
   • Loaded 75 rows from Resultat_Turnout
   • Found coordinates: Northing, Easting
   • Converting 75 coordinate pairs from SWEREF99 TM to WGS84...
   ✅ Successfully converted 75 Turnout points
   📍 Latitude between 60.510878° and 61.009065°
   📍 Longitude between 14.541326° and 15.352741°
📊 Processing Resultat_Bridge for Bridge infrastructure...
   • Loaded 25 rows from Resultat_Bridge
   • Found coordinates: Beräknad Northing, Beräknad Easting
   • Converting 25 coordinate pairs from SWEREF99 TM to WGS84...
   ✅ Successfully converted 25 Bridge points
   📍 Latitude between 60.529364° and 61.008510°
   📍 Longitude between 14.518605° and 15.295477°
📊 Processing Resultat_RailJoint for RailJoint infrastructure...
   • Loaded 335 rows from Resultat_Rai

📷 Saving railway map for documentation and analysis...
💾 Interactive railway map saved: railway_map_static.html
📄 Open railway_map_static.html in any web browser to view the complete infrastructure analysis
🎯 HTML format provides superior interactive capabilities for railway infrastructure visualization

🔍 GPS VALIDATION WITH INFRASTRUCTURE MATCHING:
Starting folder validation...

Checking folder: 2024-12-08 02-00-00 (1)
📂 Analyzing folder: 2024-12-08 02-00-00 (1)
    📡 GPS satellite data found for quality assessment
    📊 GPS satellite info: Average satellites: 4.4, Poor GPS (<4 sats): 3.6%, Very poor (<3 sats): 0.0%
    🏗️ Infrastructure coverage: 0.0%
    🎯 Infrastructure types found: []
    ❌ Enhanced GPS validation failed:
        • Mostly stationary recording: only 0.0% of points show movement

Checking folder: 2024-12-08 04-00-00 (1)
📂 Analyzing folder: 2024-12-08 04-00-00 (1)
    📡 GPS satellite data found for quality assessment
    📊 GPS satellite info: Average satellites: 3.8